# Notebook 4: The three point problem

The three point problem is a fundamental problem in geology. It is based on the fact that three non-collinear points on a plane define the orientation of the plane. The graphical solution to this problem is introduced early in Geosciences. It involves finding the strike line (a line connecting two points of equal elevation) on the plane, and the dip from two strike lines (two structure contours) on the plane.

However, there is an easier solution to this problem using linear algebra: The three points on the plane define two lines, and the cross product between these lines is parallel to the pole to the plane, from which the orientation of the plane can be estimated.

## Python function

The function below computes the strike and dip of a plane from the east, north, and up coordinates of three points on the plane:

In [ ]:
# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath("../functions"))

In [2]:
# three_points function: this function is also in functions/three_points.py

import numpy as np
from cart_to_sph import cart_to_sph
from pole import plane_from_pole

def three_points(p1,p2,p3):
	"""
	three_points calculates the strike (strike) and dip (dip)
	of a plane given the east (E), north (N), and up (U)
	coordinates of three non-collinear points on the plane
	
	p1, p2 and p3 are 1 x 3 arrays defining the location
	of the points in an ENU coordinate system. For each one
	of these arrays the first, second and third entries are 
	the E, N and U coordinates, respectively
	
	NOTE: strike and dip are returned in radians and they
	follow the right-hand rule format
	"""
	# make vectors v (p1 - p3) and u (p2 - p3)
	v = p1 - p2
	u = p2 - p3
	# take the cross product of v and u
	vcu = np.cross(v,u)
	
	# make this vector a unit vector
	mvcu = np.linalg.norm(vcu) # magnitude of the vector
	if mvcu == 0: # If points collinear
		raise ValueError("Error: points are collinear")
	
	vcu = vcu/mvcu # unit vector
	
	# make the pole vector in NED coordinates
	p = np.array([vcu[1], vcu[0], -vcu[2]])
	
	# Make pole point downwards
	if p[2] < 0:
		p *= -1.0
	
	# find the trend and plunge of the pole
	trd, plg = cart_to_sph(p[0],p[1],p[2])
	
	# find strike and dip of plane
	strike, dip = plane_from_pole(trd, plg)
	
	return strike, dip

## Application

Let's use this function to solve the following problem from Bennison (1990). For the topographic map below, borehole A passes through a coal seam at a depth of 50 m, and reaches a lower seam at a depth of 450 m. Boreholes B and C reach the lower seam at depths of 150 and 250 m, respectively.

Determine the strike and dip of the seams. Assume the seams are conformable and have a constant vertical separation of 400 m. Also, set the origin of the east-north coordinates at the lower left corner of the map.

<img src="../figures/three_point.png" alt="three_point" width="500" style="display: block; margin: 0 auto"/><br><br>

The elevation of the lower seam at A, B and C is:

A = 1000 - 450 = 550 m

B = 800 - 150 = 650 m

C = 700 - 250 = 450 m

And since the vertical distance between the seams is 400 m, the elevation of the upper seam at A, B and C is 950, 1050 and 850 m, respectively.

With the origin of the east-north coordinates at the lower left corner of the map, the coordinates of the seams at the three boreholes are:

<div align="center">

| Borehole | Lower seam | Upper seam |
| -------- | ---------- | ---------- |
| A | [393, 2374, 550] | [393, 2374, 950] |
| B | [1891, 2738, 650] | [1891, 2738, 1050] |
| C | [2191, 1037, 450] | [2191, 1037, 850] |

</div>

We now have all the information to solve this problem:


In [3]:
# lower seam
p1 = np.array([393, 2374, 550])
p2 = np.array([1891, 2738, 650])
p3 = np.array([2191, 1037, 450])

strike, dip = three_points(p1,p2,p3)
print(f"Lower seam: strike = {np.degrees(strike):.2f}, dip = {np.degrees(dip):.2f}")

# the upper seam has the same orientation
# let's check it
p1 = np.array([393, 2374, 950])
p2 = np.array([1891, 2738, 1050])
p3 = np.array([2191, 1037, 850])

strike, dip = three_points(p1,p2,p3)
print(f"Upper seam: strike = {np.degrees(strike):.2f}, dip = {np.degrees(dip):.2f}")

Lower seam: strike = 106.45, dip = 7.37
Upper seam: strike = 106.45, dip = 7.37
